In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D, Conv3D
from tensorflow.keras.layers import MaxPooling2D, MaxPooling3D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.losses import binary_crossentropy
from sklearn.metrics import f1_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.applications.vgg16 import preprocess_input

from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image 
import seaborn as sns
import os
import re
import glob
import cv2
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
import tqdm
from numpy import loadtxt
from os import *
from sklearn.utils import class_weight


Using TensorFlow backend.


In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

In [3]:
def draw_confusion_matrix(true,preds):
    conf_matx = confusion_matrix(true, preds)
    sns.heatmap(conf_matx, annot=True,annot_kws={"size": 12},fmt='g', cbar=False, cmap=plt.cm.Blues) #'viridis'
    #plt.savefig('/home/jovyan/conf_matrix.png')
    plt.show()
    
    return conf_matx

In [4]:
def plot_history(model_history, model_name):
    fig = plt.figure(figsize=(15,5), facecolor='w')
    ax = fig.add_subplot(121)
    ax.plot(model_history.history['loss'])
    ax.plot(model_history.history['val_loss'])
    ax.set(title=model_name + ': Model loss', ylabel='Loss', xlabel='Epoch')
    ax.legend(['Train', 'Val'], loc='upper left')
    ax = fig.add_subplot(122)
    ax.plot(model_history.history['accuracy'])
    ax.plot(model_history.history['val_accuracy'])
    ax.set(title=model_name + ': Model Accuracy; test='+ str(np.round(model_history.history['val_accuracy'][-1], 3)),
           ylabel='Accuracy', xlabel='Epoch')
    ax.legend(['Train', 'Val'], loc='upper left')
    #plt.savefig('/home/jovyan/curve.png')
    plt.show()
    
    return fig

In [5]:
def resize(x):
    rescaled = []

    for i in x:

        scale_percent = 140 # percent of original size
        width = int(i.shape[1] / (scale_percent / 100))
        height = int(i.shape[0] / (scale_percent / 100))
        dim = (width, height)
        resized = cv2.resize(i, dim, interpolation = cv2.INTER_LANCZOS4)
        rescaled.append(resized)

    x_orig = np.reshape( rescaled, (len( rescaled), resized.shape[1], resized.shape[1], 3))

    return x_orig

In [6]:
def loadImages(path_data):
    
    p = '/home/jovyan/DATA_MASTER_PROJECT/Check_DIFF_T0_T97/'
    
    
    
    pa_adr = p + 'ADR_cropped/'
    
    pa_control = p + 'CONTROL_cropped/'
    
    pa_hrh = p + 'HRH_cropped/'
    
    image_list = []
    
    
       


    for filename in sorted(path_data, key=natural_keys): 
        
        if 'adr' in filename:
            
            im=cv2.imread(pa_adr + filename)

            imarray = np.array(im)
            

            image_list.append(imarray)
            
        if 'control' in filename:
            
            im=cv2.imread(pa_control + filename)

            imarray = np.array(im)
            

            image_list.append(imarray)
            
        if 'hrh' in filename:
            
            im=cv2.imread(pa_hrh + filename)

            imarray = np.array(im)
            

            image_list.append(imarray)



    x_orig = np.reshape(image_list, (len(image_list), 90, 90, 3))

    return x_orig

In [7]:
def return_count(x):
    name_wel = []
    for i in sorted(x, key = natural_keys):
        name_wel.append(i.split('_')[0])

    z = sorted(list(set(name_wel)))
    r = list(range(len(z)))

    num = []
    for iz in range(len(z)):
        count = 0
        for i in sorted(x, key=natural_keys):
            if z[iz] in i:
                count += 1
        num.append(count)
    return list(zip(z, r, num))

In [8]:
def creat_label(y):
    labels = []
    for ix, _ in enumerate(y):
        
        if y[ix][0] == 'adr':
        
            labels.append([[y[ix][0],0]] * y[ix][2])
        
        if y[ix][0] == 'hrh':
            
            labels.append([[y[ix][0],1]] * y[ix][2])
            
        if y[ix][0] == 'control':
            labels.append([[y[ix][0],0]] * y[ix][2])
    
    ler = [i for sub in labels for i in sub ]
    
    _, lab= zip(*ler)

    
    return lab

In [9]:
def make_labels(data_set):
    fe = return_count(data_set)
    leb = creat_label(fe)
    y = np.array(list(leb))
    return y
    

In [10]:
def time_step_acc(tes_data,x):

    results = []            

    x_test = loadImages(tes_data)
    y_test = make_labels(tes_data)
    x_test = resize(x_test)
    x_test = preprocess_input(x_test)

    scores = x.evaluate(x_test, y_test, verbose = 0)
    results.append(scores[1]*100)

    return results

In [11]:
tot_results_accuracy = []

In [12]:
mid = ['C6', 'F11']
leb = ['D11', 'G4']
mep = ['F2', 'G10']
met = ['G5', 'B10']
oxy = ['G3', 'B8']

cyc = ['E4', 'G6']
dox = ['G8', 'D10']
olo = ['E7', 'B7']
ket = ['E10', 'B11']
orp = ['D8', 'B2']

In [13]:
#tot_well = [mid, leb, mep, met, oxy]
tot_well = [cyc, dox, olo, ket, orp]

In [14]:
#string_well = ['mid', 'leb', 'mep', 'met', 'oxy']
string_well = ['cyc', 'dox', 'olo', 'ket', 'orp']

In [ ]:

for index_t_well, _ in tqdm.tqdm(enumerate(tot_well)):

    time_points = list(map(str, range(1,97,3)))

    new_time = []
    for i in time_points:
        r = '_' + i + '.'
        new_time.append(r)
        


    path_test = '/home/jovyan/DATA_MASTER_PROJECT/Check_DIFF_T0_T97/HRH_cropped/'

    # NAME OF THE WELLS CORRESPONDING TO THE FRUG THAT YOU WANT IN THE TEST SET 

    wells_drug = [tot_well[index_t_well][0], tot_well[index_t_well][1]] 

    test = []

    for _,_, filenames in os.walk(path_test):

        for filename in sorted(filenames, key = natural_keys):

            for w in wells_drug:
                for t in new_time:
                    if '{}'.format(w) in filename and '{}tiff'.format(t) in filename:
                        test.append(filename)

    groups_list = ['CONTROL', 'HRH']#, 'CONTROL']

    fileds_of_view = ['1','2','3','4']

    field_train, field_val = train_test_split(fileds_of_view, test_size=0.2, random_state=int(np.random.randint(1,1000,1)))


    train = []

    validation = []

    group_compounds = []

    for group in tqdm.tqdm(groups_list):

        pa = '/home/jovyan/DATA_MASTER_PROJECT/Check_DIFF_T0_T97/{}_cropped/'.format(group)

        for _,_, filenames in os.walk(pa):

            for filename in sorted(filenames, key = natural_keys):

                for t in new_time:

                    if '_{}-'.format(wells_drug[0]) not in filename and '_{}-'.format(wells_drug[1]) not in filename and '{}tiff'.format(t) in filename:

                        group_compounds.append(filename)






    for i in group_compounds:

        for f in field_train:
            if '-{}_'.format(f) in i:
                train.append(i)


        for v in field_val:
            if '-{}_'.format(v) in i:
                validation.append(i)


    x_train = loadImages(train)
    y_train = make_labels(train)



    x_val = loadImages(validation)
    y_val = make_labels(validation)



    x_train = resize(x_train)


    x_val = resize(x_val)


    weights = class_weight.compute_class_weight('balanced', np.unique(y_train),y_train)


    x_train = preprocess_input(x_train)

    x_val = preprocess_input(x_val)


    es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=3)

    pretrained_model = VGG16(weights='imagenet',include_top=False, input_shape=(64, 64, 3))

    base_model = Model(inputs=pretrained_model.input, outputs=pretrained_model.get_layer('block3_pool').output)

    batch_size = 128

    datagen = ImageDataGenerator()

    datagen.fit(x_train)

    train_gen = datagen.flow(x_train, y_train,batch_size=batch_size )

    dat_val = ImageDataGenerator()

    dat_val.fit(x_val)

    val_gen = dat_val.flow(x_val, y_val,batch_size=batch_size)

    m4 = Sequential()
    m4.add(base_model)


    m4.add(BatchNormalization())
    m4.add(GlobalAveragePooling2D())
    m4.add(BatchNormalization())
    m4.add(Activation('relu'))
    m4.add(Dense(64, activation='relu'))
    m4.add(Dense(1,activation='sigmoid'))


    base_model.trainable = False

    opt = keras.optimizers.Adam(lr=1e-4)

    m4.compile(loss= keras.losses.binary_crossentropy, optimizer=opt, metrics = ['accuracy'])



    epochs = 300

    m4_h = m4.fit(train_gen,
                    steps_per_epoch=(len(x_train)/batch_size),
                    callbacks = [es],
                    epochs=epochs,
                    validation_data = (val_gen), 
                    validation_steps = (len(x_val)/batch_size),
                    class_weight = weights,
                     verbose = 1)

    base_model.trainable = True

    opt = keras.optimizers.Adam(lr=1e-5)

    m4.compile(loss= keras.losses.binary_crossentropy, optimizer=opt, metrics = ['accuracy'])

    epochs = 300

    m4_h = m4.fit(train_gen,
                    steps_per_epoch=(len(x_train)/batch_size),
                    callbacks = [es],
                    epochs=epochs,
                    validation_data = val_gen, 
                    validation_steps = (len(x_val)/batch_size),
                    class_weight = weights,
                    verbose = 1)

    l = []
    for t in new_time:
        for i in test:
            if t in i:
                l.append((i))


    grouped = {}
    for elem in l:
        key = elem.split('.tiff')[0].split('_')[5]
        grouped.setdefault(key, []).append(elem)
    grouped = grouped.values()

    test_data = list(grouped)

    r = []

    for ix ,_ in enumerate(test_data):
        r.append(time_step_acc(test_data[ix],m4))

    plt.plot(time_points,r)
    plt.savefig('/home/jovyan/{}_accuracy.png'.format(string_well[index_t_well]))
    
    tot_results_accuracy.append(r)

    p = '/home/jovyan/save_model_final/'
    m4.save(p + "{}.h5".format(string_well[index_t_well]))

0it [00:00, ?it/s]
100%|██████████| 2/2 [00:08<00:00,  4.14s/it]


In [ ]:

for i in tot_results_accuracy:
    plt.plot(time_points, i)
    plt.show

In [ ]:
mena_acc = []

for i in tot_results_accuracy:
    mena_acc.append(np.mean(i))

In [ ]:
list(zip(mena_acc,string_well))

In [ ]:
mean_acc_time = []
std_acc_time = []
for ix,_ in enumerate(time_points):

    e = [item[ix] for item in tot_results_accuracy]
    
    r = np.mean(e)
    
    s = np.std(e)
    
    std_acc_time.append(s)
    
    mean_acc_time.append(r)

In [ ]:
std_acc_time

In [ ]:
std_low  = np.subtract(mean_acc_time, std_acc_time)
std_high = np.add(mean_acc_time, std_acc_time)

In [ ]:
plt.rc('ytick', labelsize=18)
ax = plt.figure(figsize=(13,8), facecolor='w').gca()


#ax.fill_between(std_acc_time + 1, std_acc_time - 1, alpha=0.2)
ax.plot(time_points, mean_acc_time)
ax.plot(time_points,std_low)
ax.plot(time_points,std_high)
plt.show
plt.close